In [1]:
#@title 1. 의존성 패키지 설치
!pip install -q langchain langchain-openai ddgs yfinance

# langchain-community 최신 버전 설치 (ddgs 패키지 지원을 위하여)
!pip install -q --force-reinstall git+https://github.com/langchain-ai/langchain-community.git@main#subdirectory=libs/community

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
cupy-cuda12x 13.3.0 requires numpy<2.3,>=1.22, but you have numpy 2.3.2 which is incompatible.
opencv-contrib-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 2.3.2 which is incompatible.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.3.2 which is incompatible.
numba 0.60.0 r

In [2]:
import os

from langchain.tools import tool
from langchain_community.tools import DuckDuckGoSearchRun
from google.colab import userdata
import yfinance as yf

In [3]:
OPENAI_API_KEY = userdata.get("OPENAI_API_KEY")

In [4]:
#@title 2. 도구 1: 최신 뉴스 검색 도구
@tool
def news_search_tool(query: str) -> str:
    """특정 회사와 관련된 최신 뉴스 기사 3개를 검색하고 요약할 때 사용합니다.
    기업의 최신 동향, 신제품 발표, 시장 평가 등 정성적인 정보를 얻는 데 사용해야 합니다."""

    # DuckDuckGoSearchRun을 사용하여 뉴스 검색 실행
    search = DuckDuckGoSearchRun(max_results=3)
    return search.run(query)

In [5]:
#@title 3. 도구 2: 주가 데이터 조회 도구
@tool
def stock_price_tool(ticker: str) -> str:
    """특정 회사의 주식 티커(ticker)를 사용하여 현재 주가, 변동률, 거래량 등 정량적인 데이터를 조회할 때 사용합니다.
    입력은 반드시 'NVDA', 'MSFT', 'GOOGL'과 같은 주식 티커 형식이어야 합니다."""

    try:
        stock = yf.Ticker(ticker)
        # 현재 주가 정보를 가져옴
        hist = stock.history(period="1d")
        if hist.empty:
            return f"'{ticker}'에 대한 데이터를 찾을 수 없습니다. 티커가 올바른지 확인해주세요."

        current_price = hist['Close'].iloc[-1]
        previous_close = hist['Open'].iloc[-1]
        change = current_price - previous_close
        change_percent = (change / previous_close) * 100

        # 결과를 문자열로 포맷팅하여 반환
        return (
            f"[{ticker}] 현재 주가: ${current_price:.2f} (전일 대비 ${change:.2f}, {change_percent:.2f}%)\n"
            f"시가: ${hist['Open'].iloc[-1]:.2f}, 고가: ${hist['High'].iloc[-1]:.2f}, 저가: ${hist['Low'].iloc[-1]:.2f}\n"
            f"거래량: {hist['Volume'].iloc[-1]:,}"
        )
    except Exception as e:
        return f"주가 조회 중 오류가 발생했습니다: {e}"

In [6]:
#@title 4. 최종 도구 리스트
tools = [news_search_tool, stock_price_tool]


print("주식 트렌드 분석 에이전트를 위한 도구 2개 정의 완료.")
for t in tools:
    print(f"- {t.name}: {t.description}")

주식 트렌드 분석 에이전트를 위한 도구 2개 정의 완료.
- news_search_tool: 특정 회사와 관련된 최신 뉴스 기사 3개를 검색하고 요약할 때 사용합니다.
    기업의 최신 동향, 신제품 발표, 시장 평가 등 정성적인 정보를 얻는 데 사용해야 합니다.
- stock_price_tool: 특정 회사의 주식 티커(ticker)를 사용하여 현재 주가, 변동률, 거래량 등 정량적인 데이터를 조회할 때 사용합니다.
    입력은 반드시 'NVDA', 'MSFT', 'GOOGL'과 같은 주식 티커 형식이어야 합니다.


In [7]:
#@title 5. 에이전트 구성
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_openai import ChatOpenAI

# 이전 단계에서 정의한 tools 리스트가 있다고 가정
# from project3_tools import tools, llm

# 에이전트의 역할(금융 분석가)과 최종 목표(분석 보고서 작성)를 시스템 메시지에 정의
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
     당신은 유능한 금융 분석가입니다. 당신의 임무는 주어진 회사에 대한 최신 뉴스와 주가 데이터를 수집하고,
     이를 종합하여 투자자를 위한 간결한 분석 보고서를 작성하는 것입니다.

     보고서 작성 프로세스는 다음과 같습니다:
     1. 먼저 'news_search_tool'을 사용하여 회사의 최신 뉴스를 검색하고 주요 동향을 파악합니다.
     2. 다음으로 'stock_price_tool'을 사용하여 현재 주가 데이터를 확인합니다. (만약 주식 티커를 모른다면, 다른 도구를 사용해 먼저 찾아야 합니다.)
     3. 마지막으로, 수집된 모든 정보(뉴스 동향, 주가 데이터)를 종합하여 아래 형식에 맞춰 최종 '분석 보고서'를 작성합니다.

     최종 답변 형식:
     ## [회사명] 주식 트렌드 분석 보고서
     ### 1. 최근 주요 뉴스 요약
     - (뉴스 1 요약)
     - (뉴스 2 요약)
     ...
     ### 2. 현재 주가 동향
     - (주가 데이터 요약 및 분석)
     ### 3. 종합 분석
     - (뉴스 내용과 주가 데이터를 연관지어 종합적인 분석 의견 제시)
     """),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),

])

llm = ChatOpenAI(
    model="gpt-5",
    api_key=OPENAI_API_KEY,
)
agent = create_openai_tools_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [8]:
#@title 6. 에이전트 실행
print("--- 'NVIDIA'에 대한 분석 보고서 생성 시작 ---")
response = agent_executor.invoke({"input": "NVIDIA의 최신 주식 동향을 분석해줘."})

print("\n" + "="*50)
print("              최종 생성된 보고서")
print("="*50 + "\n")
print(response['output'])

--- 'NVIDIA'에 대한 분석 보고서 생성 시작 ---


> Entering new AgentExecutor chain...

Invoking: `news_search_tool` with `{'query': 'NVIDIA 최신 뉴스 NVDA'}`


1 week ago - 엔비디아 코퍼레이션( Nvidia Corporation)은 캘리포니아주 샌타클래라에 본사를 두고 델라웨어에 법인을 둔 미국의 다국적 기업이자 기술 회사이다. 데이터 사이언스 및 고성능 컴퓨팅을 위한 그래픽 처리 장치(GPU), API(애플리... May 29, 2025 - Nvidia reported better-than-expected earnings and revenue on Wednesday. May 14, 2025 - 해당 칩은 대규모 데이터 센터 프로젝트에 사용될 예정 이다. 이 소식은 Nvidia (NASDAQ:NVDA) 주가가 이미 약 11% 급등한 한 주에 이어 나왔다. 2 weeks ago - NVIDIA (NASDAQ:NVDA) has scheduled its second-quarter fiscal 2026 financial results conference call for August 27, 2025, at 2 p.m. PT (5 p.m. ET). The company will release written commentary from CFO Colette Kress at approximately 1:20 p.m. 2 weeks ago - Stay updated with the latest news on NVDA stock. Get breaking updates, market trends on NVIDIA financial performance.
Invoking: `stock_price_tool` with `{'ticker': 'NVDA'}`


[NVDA] 현재 주가: $177.99 (전일 대비 $5.38, 3.12%)
시가: $172.61, 고가: $178.59,

In [17]:
!pip install -q google-genai pydantic langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.4/49.4 kB 536.7 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 5.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [18]:
import json

from google import genai
from google.genai import types
from pydantic import BaseModel, Field

In [19]:
GEMINI_API_KEY = userdata.get("GEMINI_API_KEY")
gemini_client = genai.Client(api_key=GEMINI_API_KEY)

In [20]:
class VerificationPlan(BaseModel):
    """보고서 초안의 주장을 검증하기 위한 질문 목록을 담는 스키마."""
    questions: list[str] = Field(description="검증할 사실 관계 질문들의 목록")

In [21]:
#@title 7. 신뢰성 개선을 위한 구글 검색 네이티브 그라운딩 설정

# 구글 검색 그라운딩 도구
google_search_tool = types.Tool(google_search=types.GoogleSearch())
grounding_config = types.GenerateContentConfig(tools=[google_search_tool])


# 주가 데이터 조회 도구
def stock_price_tool(ticker: str) -> str:
    """특정 회사의 주식 티커를 사용하여 현재 주가 데이터를 조회합니다."""
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1d")
        if hist.empty: return f"'{ticker}' 데이터를 찾을 수 없습니다."
        current_price = hist['Close'].iloc[-1]
        previous_close = hist['Open'].iloc[-1]
        change = current_price - previous_close
        change_percent = (change / previous_close) * 100
        return (
            f"[{ticker}] 현재 주가: ${current_price:.2f} (전일 대비 ${change:.2f}, {change_percent:.2f}%)\n"
            f"시가: ${hist['Open'].iloc[-1]:.2f}, 고가: ${hist['High'].iloc[-1]:.2f}, 저가: ${hist['Low'].iloc[-1]:.2f}\n"
            f"거래량: {hist['Volume'].iloc[-1]:,}"
        )
    except Exception as e:
        return f"주가 조회 중 오류 발생: {e}"

In [22]:
#@title 8. CoVe 워크플로우 구성
def run_cove_analysis(company: str, ticker: str):
    """주어진 회사와 티커에 대해 CoVe 워크플로우를 실행합니다."""

    # [1단계] 초기 답변 생성
    print("--- [CoVe 1/4] 초기 답변 생성 중 ---")
    stock_data = stock_price_tool(ticker)
    draft_prompt = f"""
    {company}의 최신 주식 동향을 분석하는 보고서를 간결하게 작성해주세요.

    보고서에는 해당 기업의 최신 뉴스 정보가 포함되어야 하며 그 뉴스는 신뢰할 수 있는
    대중적이고 검증된 미디어 소스여야 합니다.

    그 콘텐츠가 신뢰성이 있는 이유와 근거 사항을 선행 기술하시고, 콘텐츠도 함께 출력해주세요.

    현재 해당 기업의 주가 정보는 다음과 같습니다:
    {stock_data}
    """.strip()
    draft_response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=draft_prompt,
        config=grounding_config,
    )
    draft_report = draft_response.text
    print("✓ 초기 보고서 초안 생성 완료.")

    # [2단계] 검증 계획 수립 (검증 질문 생성)
    print("\n--- [CoVe 2/4] 검증 계획 수립 중 ---")
    plan_prompt = f"""
    다음은 {company}에 대한 분석 보고서 초안입니다:
    ---
    {draft_report}
    ---
    위 보고서 초안에 포함된 핵심 주장들의 신뢰도를 한 번 더 확인하기 위한, 의미 있는 검증 질문 목록을 작성해주세요.
    각 지식의 요소를 원자적인 주장 단위로 나누고 검증을 위한 질문을 문장 단위로 정리하여 다음의 CoVe 체인을 염두해두어야 합니다.
    """.strip()

    plan_response = gemini_client.models.generate_content(
        model="gemini-2.5-flash",
        contents=plan_prompt,
        config={
            "response_mime_type": "application/json",
            "response_schema": VerificationPlan,
        },
    )

    plan: VerificationPlan = plan_response.parsed
    verification_questions = plan.questions
    print("✓ 생성된 검증 질문:\n", verification_questions)

    # [3단계] 검증 실행 (Google Search 그라운딩 사용)
    print("\n--- [CoVe 3/4] 검증 실행 중 ---")
    verification_results = []
    for question in verification_questions:
        print(f"  - 검증 질문 실행: {question}")
        verification_response = gemini_client.models.generate_content(
            model="gemini-2.5-flash", contents=question, config=grounding_config
        )
        answer = verification_response.text.strip()
        print(f"  - 검증 질문 결과: {answer}")
        verification_results.append({"question": question, "answer": answer})
    print("✓ 검증 실행 완료.")

    # [4단계] 최종 답변 수정 및 생성
    print("\n--- [CoVe 4/4] 최종 보고서 생성 중 ---")
    final_prompt = f"""
    당신은 매우 신중한 금융 분석가입니다.
    아래의 [초기 보고서 초안]과 [검증 결과]를 종합적으로 검토해서, 최종적으로 사실에 기반한 분석 보고서를 작성해주십시요.

    [검증 결과]를 바탕으로 [초기 보고서 초안]의 모든 주장을 비판적으로 평가하고,
    오류가 있거나 근거가 부족하면 반드시 수정하거나 삭제해야 합니다.

    [초기 보고서 초안]:
    {draft_report}

    [검증 결과]:
    {json.dumps(verification_results, ensure_ascii=False, indent=2)}

    최종 분석 보고서:
    """
    final_response = gemini_client.models.generate_content(
        model="gemini-2.5-pro", contents=final_prompt
    )
    print("✓ 최종 보고서 생성 완료.")
    return final_response.text

In [23]:
#@title 9. 최종 CoVe 워크플로우 실행
user_input = {"company": "NVIDIA", "ticker": "NVDA"}
final_report = run_cove_analysis(user_input["company"], user_input["ticker"])

print("\n" + "="*50)
print("             최종 생성된 보고서 (CoVe 적용 후)")
print("="*50 + "\n")
print(final_report)

--- [CoVe 1/4] 초기 답변 생성 중 ---
✓ 초기 보고서 초안 생성 완료.

--- [CoVe 2/4] 검증 계획 수립 중 ---
✓ 생성된 검증 질문:
 ['2025년 8월 25일 기준 NVIDIA(NVDA)의 현재 주가가 177.99달러인가요?', '2025년 8월 25일 기준 NVIDIA(NVDA)의 주가가 전일 대비 5.38달러(3.12%) 상승했나요?', '2025년 8월 25일 기준 NVIDIA(NVDA)의 시가가 172.61달러인가요?', '2025년 8월 25일 기준 NVIDIA(NVDA)의 고가가 178.59달러인가요?', '2025년 8월 25일 기준 NVIDIA(NVDA)의 저가가 171.20달러인가요?', '2025년 8월 25일 기준 NVIDIA(NVDA)의 거래량이 172,334,200주인가요?', 'NVIDIA는 인공지능(AI) 칩 시장의 선두 주자인가요?', 'NVIDIA의 최신 기술 발전과 파트너십 발표가 주가에 긍정적인 영향을 미치고 있나요?', "NVIDIA가 차세대 AI 칩 'Blackwell' 아키텍처 기반의 GB200 출시 계획을 밝혔나요?", 'GB200 출시 계획이 AI 인프라 시장에서 NVIDIA의 리더십을 공고히 할 것이라는 기대를 모으고 있나요?', 'GB200이 데이터센터의 AI 성능을 크게 향상시킬 것으로 예상되나요?', 'GB200이 클라우드 서비스 제공업체와 대기업들의 수요를 자극할 것으로 전망되나요?', 'NVIDIA의 신제품 출시가 장기적인 성장 동력으로 작용하며 투자자들의 긍정적인 평가를 받고 있나요?', 'NVIDIA는 자사의 소프트웨어 플랫폼인 CUDA 생태계를 통해 AI 개발자 커뮤니티 내에서 독점적인 위치를 유지하고 있나요?', 'CUDA 생태계가 경쟁사들이 쉽게 따라잡기 어려운 강력한 해자(moat)로 작용하나요?', 'CUDA 생태계가 하드웨어 판매를 넘어선 소프트웨어 및 서비스 수익 창출 가능성을 높이고 있나요?', 'NVIDIA의 최근 발표된 분기 실적이 시장의 기대를 뛰어

In [24]:
#@title 10. 자율 에이전트로 재구성
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import AgentExecutor, create_tool_calling_agent


# 에이전트가 사용할 수 있도록, 네이티브 SDK 호출 로직을 LangChain의 @tool로 래핑
@tool
def reliable_news_search_tool(query: str) -> str:
    """회사의 최신 기술 동향, 재무 실적과 관련된 신뢰성 높은 뉴스를 검색할 때만 사용합니다.
    이 도구는 Gemini 모델의 네이티브 Google Search 그라운딩 기능을 사용합니다."""
    print(f"--- [Tool Executing] reliable_news_search_tool with query: {query} ---")
    try:
        google_search_tool = types.Tool(google_search=types.GoogleSearch())
        grounding_config = types.GenerateContentConfig(tools=[google_search_tool])
        model = "gemini-2.5-flash"
        response = gemini_client.models.generate_content(
            model=model, contents=query, config=grounding_config
        )
        return response.text
    except Exception as e:
        return f"뉴스 검색 중 오류 발생: {e}"


@tool
def stock_price_tool(ticker: str) -> str:
    """특정 회사의 주식 티커를 사용하여 현재 주가 데이터를 조회합니다. 입력은 반드시 'NVDA', 'MSFT'와 같은 주식 티커 형식이어야 합니다."""
    print(f"--- [Tool Executing] stock_price_tool with ticker: {ticker} ---")
    # (내용은 이전과 동일)
    try:
        stock = yf.Ticker(ticker)
        hist = stock.history(period="1d")
        if hist.empty: return f"'{ticker}' 데이터를 찾을 수 없습니다."
        current_price = hist['Close'].iloc[-1]; previous_close = hist['Open'].iloc[-1]
        change = current_price - previous_close; change_percent = (change / previous_close) * 100
        return (f"[{ticker}] 현재 주가: ${current_price:.2f} (전일 대비 ${change:.2f}, {change_percent:.2f}%)\n"
                f"시가: ${hist['Open'].iloc[-1]:.2f}, 고가: ${hist['High'].iloc[-1]:.2f}, 저가: ${hist['Low'].iloc[-1]:.2f}\n"
                f"거래량: {hist['Volume'].iloc[-1]:,}")
    except Exception as e: return f"주가 조회 중 오류 발생: {e}"

tools = [reliable_news_search_tool, stock_price_tool]

# --- 4. LangChain 자율 에이전트 구성 ---
llm = ChatGoogleGenerativeAI(
    google_api_key=GEMINI_API_KEY,
    model="gemini-2.5-flash",
    temperature=0,
)

# [개선된] CoVe 논리를 내재화한 에이전트 시스템 프롬프트
agent_prompt = ChatPromptTemplate.from_messages([
    ("system", """
    당신은 신중하고 유능한 금융 분석가입니다. 당신의 임무는 주어진 회사에 대한 분석 보고서를 작성하는 것입니다.
    당신은 반드시 다음의 정해진 절차에 따라 순서대로 행동해야 합니다:

    1.  **뉴스 검색**: 먼저 `reliable_news_search_tool` 도구를 사용하여 회사의 최신 기술 동향, 재무 실적과 관련된 신뢰할 수 있는 뉴스를 검색합니다. 검색어는 ' [회사명] 최신 기술 뉴스 및 재무 실적'과 같이 구체적이어야 합니다.
    2.  **주가 조회**: 다음으로 `stock_price_tool`을 사용하여 회사의 현재 주가 데이터를 조회합니다. (만약 주식 티커를 모른다면, 첫 단계에서 얻은 뉴스를 바탕으로 추론하거나 `reliable_news_search_tool`을 다시 사용해 먼저 찾아야 합니다.)
    3.  **종합 및 보고서 작성**: 위 두 단계에서 수집된 모든 정보가 확보되면, 더 이상 도구를 사용하지 말고 수집된 정보를 종합하여 아래 형식에 맞춰 최종 '분석 보고서'를 작성합니다.

    최종 답변 형식:
    ## [회사명] 주식 트렌드 분석 보고서
    ### 1. 최근 주요 뉴스 요약
    ...
    ### 2. 현재 주가 동향
    ...
    ### 3. 종합 분석
    ...
    """),
    ("user", "{input}"),
    MessagesPlaceholder(variable_name="agent_scratchpad"),
])

# create_tool_calling_agent는 OpenAI 뿐만 아니라 Gemini와 같은 최신 Tool-Calling 모델과 호환된다.
agent = create_tool_calling_agent(llm, tools, agent_prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

# --- 5. 자율 에이전트 실행 ---
print("--- [LangChain 자율 에이전트] 'NVIDIA'에 대한 분석 보고서 생성 시작 ---")
response = agent_executor.invoke({"input": "NVIDIA의 최신 주식 동향을 분석해줘."})

print("\n" + "="*50)
print("             최종 생성된 보고서 (자율 에이전트)")
print("="*50 + "\n")
print(response['output'])

--- [LangChain 자율 에이전트] 'NVIDIA'에 대한 분석 보고서 생성 시작 ---


> Entering new AgentExecutor chain...

Invoking: `reliable_news_search_tool` with `{'query': 'NVIDIA 최신 기술 뉴스 및 재무 실적'}`


--- [Tool Executing] reliable_news_search_tool with query: NVIDIA 최신 기술 뉴스 및 재무 실적 ---
## NVIDIA, 최신 AI 칩 및 로보틱스 혁신 선도 속 견조한 재무 실적

엔비디아가 인공지능(AI) 칩 시장에서의 지배력을 강화하는 최신 기술을 지속적으로 선보이고 있으며, 특히 중국 시장을 겨냥한 새로운 AI 칩 개발과 로보틱스 분야의 혁신적인 플랫폼 공개로 주목받고 있습니다. 이와 함께 엔비디아는 꾸준한 매출 성장과 높은 순이익을 기록하며 견조한 재무 실적을 유지하고 있습니다.

### 최신 기술 뉴스: 중국 시장 맞춤형 AI 칩 및 로보틱스 혁신 가속화

엔비디아는 최근 미·중 무역 갈등 속에서도 중국 시장에 AI 칩을 공급하기 위한 노력을 지속하고 있습니다. 현재 중국에 판매가 허용된 AI 칩보다 더 강력한 성능을 지닌 새로운 AI 칩인 'B30A'를 개발 중이라고 로이터 통신이 보도했습니다. 'B30A'는 엔비디아의 최신 아키텍처인 블랙웰(Blackwell)을 기반으로 하며, 단일 다이(single-die) 설계를 채택하여 주력 제품인 B300 가속기 카드의 절반 수준의 연산 성능을 제공할 것으로 예상됩니다. 이 칩에는 고대역폭 메모리(HBM)와 프로세서 간 빠른 데이터 전송을 위한 엔비디아의 NV링크(NVLink) 기술이 탑재될 예정입니다. 엔비디아는 이르면 다음 달부터 중국 고객들에게 테스트용 샘플을 공급하기를 희망하고 있다고 알려졌습니다.

또한, 엔비디아는 블랙웰 기반의 AI 추론 작업을 위해 설계된 또 다른 중국 전용 칩인 'RTX6000D'를 9월부터 중국 고객들에게